In [5]:
import os
from ultralytics import YOLO
import cv2
import xgboost as xgb
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime

def send_alert_email(frame_number, timestamp):
    # Email configuration
    sender_email = "chotu.saket.raj@gmail.com"  # Replace with your email
    receiver_email = "saket.sraj@gmail.com"
    password = "zkki umbj tdhk ayjr"  # Use app-specific password for Gmail
    
    # Create message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = "Suspicious Activity Detected - Possible Shoplifting"
    
    body = f"""
    ALERT: Suspicious activity detected!
    
    Details:
    - Time: {timestamp}
    - Frame number: {frame_number}
    - Probability: High likelihood of shoplifting
    
    Please review the footage immediately.
    """
    
    message.attach(MIMEText(body, "plain"))
    
    try:
        # Connect to SMTP server (for Gmail)
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message.as_string())
        print("Alert email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

def detect_shoplifting(video_path):
    model_yolo = YOLO(r"/Users/puttu/Desktop/FYP-shoplifting/FYP-shoplifting/best.pt")
    model = xgb.Booster()
    model.load_model(r"/Users/puttu/Desktop/FYP-shoplifting/FYP-shoplifting/model_weights.json")

    cap = cv2.VideoCapture(video_path)
    print('Total Frame', cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
    
    # Generate a unique output path
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_path = fr"/Users/puttu/Desktop/FYP-shoplifting/FYP-shoplifting/output_video/{video_name}_output.mp4"
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_tot = 0
    alert_sent = False  # To prevent multiple alerts for the same event

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            results = model_yolo(frame, verbose=False)
            annotated_frame = results[0].plot(boxes=False)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            for r in results:
                bound_box = r.boxes.xyxy
                conf = r.boxes.conf.tolist()
                keypoints = r.keypoints.xyn.tolist()

                print(f'Frame {frame_tot}: Detected {len(bound_box)} bounding boxes')

                for index, box in enumerate(bound_box):
                    if conf[index] > 0.75:
                        x1, y1, x2, y2 = box.tolist()
                        data = {}

                        # Initialize the x and y lists for each possible key
                        for j in range(len(keypoints[index])):
                            data[f'x{j}'] = keypoints[index][j][0]
                            data[f'y{j}'] = keypoints[index][j][1]

                        df = pd.DataFrame(data, index=[0])
                        dmatrix = xgb.DMatrix(df)
                        cut = model.predict(dmatrix)
                        binary_predictions = (cut > 0.5).astype(int)
                        print(f'Prediction: {binary_predictions}')

                        if binary_predictions == 0:
                            conf_text = f'Suspicious ({conf[index]:.2f})'
                            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 7, 58), 2)
                            cv2.putText(annotated_frame, conf_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 7, 58), 2)
                            
                            # Send alert if not already sent for this detection
                            if not alert_sent:
                                send_alert_email(frame_tot, current_time)
                                alert_sent = True
                        else:
                            alert_sent = False  # Reset if normal activity is detected

                        if binary_predictions == 1:
                            conf_text = f'Normal ({conf[index]:.2f})'
                            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (57, 255, 20), 2)
                            cv2.putText(annotated_frame, conf_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (57, 255, 20), 2)

            cv2.imshow('Frame', annotated_frame)
            out.write(annotated_frame)
            frame_tot += 1
            print('Processed Frame:', frame_tot)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage:
video_path = r"/Users/puttu/Desktop/FYP-shoplifting/FYP-shoplifting/one.mp4"
detect_shoplifting(video_path)

📌 Loading YOLOv5 and XGBoost models...
📊 Total Frames: 6138.0
📸 Frame 0: Detected 4 bounding boxes


ValueError: feature_names mismatch: ['x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13', 'y13', 'x14', 'y14', 'x15', 'y15', 'x16', 'y16'] ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11', 'y12', 'y13', 'y14', 'y15', 'y16']